# H20 AutoML Melbourne Housing Regression 

Sanika Patne, Greeshma Tatinen, Nik Bear Brown

##**Abstract**


An abstract is:

What is the question?    
What did you do?   
What were the results?



The goal was to construct a realistic model to precisely predict the price of houses in Melbourne. It is generally believed that it’s impossible to precisely predict the price of house as it depends on various factors. For this dataset price is the dependent variable and others are independent variables. First, major factors affecting the housing prices should be selected. Then, a model for housing price prediction should be established from linear regression. That should be applied to the Melbourne housing dataset to test the model. Through the data analysis and test it can be summarized that the linear regression model can effectively predict and analyze the housing price to some extent, while the algorithm can still be improved through more advanced machine learning methods.

##**Dataset**

This is a  static snapshot of a dataset created by Tony Pino.
*(Format this so it is easy to read)*

It was scraped from publicly available results posted every week from Domain.com.au. He cleaned it well, and now it's up to you to make data analysis magic. The dataset includes Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale and distance from C.B.D.<br>


Notes on Specific Variables
Rooms: Number of rooms<br>
Price: Price in dollars<br>
Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.<br>
Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.<br>
SellerG: Real Estate Agent<br>
Date: Date sold<br>
Distance: Distance from CBD<br>
Regionname: General Region (West, North West, North, North east …etc)<br>
Propertycount: Number of properties that exist in the suburb.<br>
Bedroom2 : Scraped # of Bedrooms (from different source)<br>
Bathroom: Number of Bathrooms<br>
Car: Number of carspots<br>
Landsize: Land Size<br>
BuildingArea: Building Size<br>
CouncilArea: Governing council for the area

##**Importing required Libraries and H20 Initialization** 

(Talk a little about H20)

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.38.0.4-py2.py3-none-any.whl size=177628739 sha256=bb04f7f1f82f1e4fd6f25c70483595ff5ed800a92f0dcdb24387fd56231d94a4
  Stored in directory: /root/.cache/pip/wheels/6b/1d/65/39927b5984bde22f615b4c6e9a8ee98ea4abf1acb5926623ff
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set(
    context="notebook",
    palette="Spectral",
    style="darkgrid",
    font_scale=1.5,
    color_codes=True,
)
import warnings

warnings.filterwarnings("ignore")
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from sklearn.model_selection import train_test_split as tts
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from yellowbrick.regressor import ResidualsPlot
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
min_mem_size = 6
run_time = 222

In [ ]:
aww pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

6


In [ ]:
port_no = random.randint(5555, 55555)

#  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
try:
    h2o.init(
        strict_version_check=False, min_mem_size_GB=min_mem_size, port=port_no
    )  # start h2o
except:
    logging.critical("h2o.init")
    h2o.download_all_logs(dirname=logs_path, filename=logfile)
    h2o.cluster().shutdown()
    sys.exit(2)

Checking whether there is an H2O instance running at http://localhost:10739 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7dhig65_
  JVM stdout: /tmp/tmp7dhig65_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp7dhig65_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:10739
Connecting to H2O server at http://127.0.0.1:10739 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_unknownUser_gk0gi8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


**Importing Data**

In [ ]:
# Import the processed data from notebook One
url = (
    "https://raw.githubusercontent.com/aiskunks/YouTube/main/CSV/Melbourne_Housing.csv"
)
df = h2o.import_file(path=url)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


##**Studying the Data**

In [ ]:
df.describe()

Rows:6196
Cols:21

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
type,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,141.7333763718528,3058.9891865719806,1.931407359586829,0.5832795351839898,717.5915106520335,1.4372175597159458,97.45432537120723,25.937378954163982,71.29841833440932,71.09522272433827,2.902033570045191,0.5763395739186573,1.5735958683021307,361.07730794060683,191.0622982569399,81.70125887669462,14.678502259522269,1756.043092317624,1867.631536475145,3.8244028405422843,188.2254680438993
maxs,286.0,6116.0,7.0,2.0,1569.0,4.0,202.0,50.0,191.0,184.0,9.0,7.0,10.0,1127.0,548.0,134.0,30.0,3462.0,3761.0,7.0,283.0
sigma,83.20219111646345,1766.4944044852118,0.9710788063154112,0.8539919799572625,402.38630106987245,1.1429382882321082,57.472556631767944,14.404004145977009,38.656203305486514,46.96929056768282,0.9700548212641686,0.7113618878731119,0.929946606819003,285.75760340659457,107.81590440091563,34.35460062890252,9.217255605003533,896.9752262444393,979.3745168200843,1.9865142710876427,71.49054360008813
zeros,36,1,329,4088,1,757,1,114,4,31,5,3284,431,1015,1,1,279,1,1,571,2
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.0,2790.0,1.0,0.0,839.0,1.0,18.0,41.0,12.0,50.0,2.0,0.0,0.0,110.0,85.0,23.0,29.0,1712.0,1839.0,2.0,135.0
1,0.0,4520.0,2.0,0.0,1118.0,3.0,18.0,42.0,12.0,50.0,3.0,1.0,0.0,88.0,224.0,23.0,29.0,1690.0,1854.0,2.0,135.0
2,0.0,4882.0,3.0,0.0,1174.0,4.0,112.0,43.0,12.0,50.0,3.0,0.0,2.0,74.0,212.0,130.0,29.0,1722.0,1851.0,2.0,135.0


Write about whether these numbers make sense ... plot long and lat on a map to check where these houses are. Where there really houses built there at the birth of Christ?

I believe price is wrong as the price seems dependent on the sale date.  This needs to be fixed.



In [ ]:
df.types

{'Suburb': 'int',
 'Address': 'int',
 'Rooms': 'int',
 'Type': 'int',
 'Price': 'int',
 'Method': 'int',
 'SellerG': 'int',
 'Date': 'int',
 'Distance': 'int',
 'Postcode': 'int',
 'Bedroom2': 'int',
 'Bathroom': 'int',
 'Car': 'int',
 'Landsize': 'int',
 'BuildingArea': 'int',
 'YearBuilt': 'int',
 'CouncilArea': 'int',
 'Lattitude': 'int',
 'Longtitude': 'int',
 'Regionname': 'int',
 'Propertycount': 'int'}

In [ ]:
df.shape

(6196, 21)

In [ ]:
# Create a 80/20 train/test split
pct_rows = 0.80
df_train, df_test = df.split_frame([pct_rows])

In [ ]:
print(df_train.shape)
print(df_test.shape)

(4980, 21)
(1216, 21)


In [ ]:
# Display all columns in X
X = df.columns
print(X)

['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount']


In [ ]:
# Seperate Dependent variable from Independent variable
y_numeric = "Price"
X.remove(y_numeric)
print(X)

['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount']


In [ ]:
dfpd = h2o.import_file(path=url)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


##**H20 AutoML Execution**

In [ ]:
# Set up AutoML
aml = H2OAutoML(max_runtime_secs=run_time, seed=1)

In [ ]:
# pass the data through H20AutoML
aml.train(x=X, y=y_numeric, training_frame=df_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,91.47044,3.486643,90.99218,92.46944,86.6315,96.33705,90.922035
mean_residual_deviance,15491.213,1593.7357,15326.587,16435.021,13174.9795000,17420.75,15098.726
mse,15491.213,1593.7357,15326.587,16435.021,13174.9795000,17420.75,15098.726
null_deviance,162459424.0000000,5637426.0,162343856.0000000,164077168.0000000,159429376.0000000,170777920.0000000,155668752.0000000
r2,0.9050196,0.0087013,0.9022002,0.8978894,0.9181833,0.8976849,0.9091405
residual_deviance,15442731.0000000,1843628.4,15863017.0000000,16747287.0000000,13043229.0000000,17473012.0000000,14087111.0000000
rmse,124.32932,6.464532,123.80059,128.19914,114.78231,131.98769,122.87687
rmsle,nan,0.0,nan,nan,nan,nan,nan


In [ ]:
print(aml.leaderboard)

model_id                                                    rmse      mse      mae    rmsle    mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_1_20230203_190211     124.518  15504.8  91.4909      nan                   15504.8
StackedEnsemble_AllModels_2_AutoML_1_20230203_190211     125.043  15635.8  91.7797      nan                   15635.8
StackedEnsemble_AllModels_1_AutoML_1_20230203_190211     125.255  15688.8  92.0268      nan                   15688.8
StackedEnsemble_BestOfFamily_3_AutoML_1_20230203_190211  126.521  16007.5  93.1319      nan                   16007.5
StackedEnsemble_BestOfFamily_4_AutoML_1_20230203_190211  126.778  16072.7  93.3569      nan                   16072.7
StackedEnsemble_BestOfFamily_2_AutoML_1_20230203_190211  127.048  16141.2  93.6472      nan                   16141.2
GBM_3_AutoML_1_20230203_190211                           128.289  16458    95.06        nan                   16458
GBM_2_AutoML_1_20230203_190211                           1

##**Analysing relation between all variables**

When all independent variables were plotted against 'price' variable it is observed that for'Landsize' and 'BuildingArea' roughly a linear line is obtained. So it can be concluded from this plot that variables Landsize and BuildingArea have linear relation with Price.

VIF stands for Variance Inflation Factor.It signifies multicollinearity among set of different independent variable. A value greater than 10 means that the independent variable is highly correlated with some other variable. Will calculate VIF value for all variables to check the dependency and then will drop the variables with high VIF value.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Indicate which variables to compute VIF
Multic = dfpd[
    [
        "YearBuilt",
        "Distance",
        "Rooms",
        "Longtitude",
        "Bathroom",
        "Suburb",
        "Address",
        "Type",
        "SellerG",
        "Method",
        "Date",
        "Postcode",
        "Bedroom2",
        "Car",
        "Landsize",
        "BuildingArea",
        "CouncilArea",
        "Lattitude",
        "Regionname",
        "Propertycount",
    ]
]
# Compute VIF
vif = pd.DataFrame()
vif["variables"] = Multic.columns
vif["VIF"] = [
    variance_inflation_factor(Multic.values, i) for i in range(Multic.shape[1])
]
vif

AttributeError: ignored

VIF for 'YearBuilt','Rooms','Longtitude','Bedroom2' and 'BuildingArea' is greater than 10.'Rooms','Bedroom2' and 'BuildingArea' specify the same thing. More the rooms more will be the building area. Will drop the variables Bedroom2, Longtitude and YearBuilt. Not dropping Rooms as for a customer number of rooms is an important aspect.

Ordinary least squares (OLS) regression is a statistical method of analysis used for multivariate model that estimates the relationship between one or more independent variables and a dependent variable; the method estimates the relationship by minimizing the sum of the squares in the difference between the observed and predicted values of the dependent variable.

In [ ]:
import statsmodels.formula.api as smf  # OLS model Library

results = smf.ols(
    "Price ~ Distance + Rooms + Bathroom +Suburb + Address + Type + SellerG + Method + Date + Postcode + Car + Landsize + BuildingArea + CouncilArea + Lattitude + Regionname + Propertycount",
    data=dfpd,
).fit()
results.summary()

PatsyError: ignored

As seen from the OLS summary p-value for 'Address' and 'Date' is greater than 0.05. An extra variable 'Address' is unneccsary as 'Suburb','Postcode' and 'RegionName' will help us tell the location. Dropping them will give us a model with all variables whose p-values are less than 0.05. The accuracy for this model is 76.4%. Now will check VIF and p-values again for the selected independent variables.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Indicate which variables to compute VIF
Multic = dfpd[
    [
        "Distance",
        "Rooms",
        "Bathroom",
        "Suburb",
        "Type",
        "SellerG",
        "Method",
        "Postcode",
        "Car",
        "Landsize",
        "BuildingArea",
        "CouncilArea",
        "Lattitude",
        "Regionname",
        "Propertycount",
    ]
]
# Compute VIF
vif = pd.DataFrame()
vif["variables"] = Multic.columns
vif["VIF"] = [
    variance_inflation_factor(Multic.values, i) for i in range(Multic.shape[1])
]
vif

AttributeError: ignored

In [ ]:
import statsmodels.formula.api as smf  # OLS model Library

results = smf.ols(
    "Price ~ Distance + Rooms + Bathroom +Suburb  + Type + SellerG + Method  + Postcode + Car + Landsize + BuildingArea + CouncilArea + Lattitude + Regionname + Propertycount",
    data=dfpd,
).fit()
results.summary()

VIF for all except two variables are less than 10. Dropping either of the two variables 'Rooms' or 'BuildingArea' wont make sense as a customer should know the number of rooms and BuildingArea.P-values for all the variables are less than 0.05 and the accuracy is 76.4%. It is unaffected even after removing 'Address' and 'Date' tells us that these two variables were not important for calculating 'Price'.



In [ ]:
dfpd1 = dfpd.drop(["Address", "Date", "Bedroom2", "Longtitude", "YearBuilt"], axis=1)

In [ ]:
dfpd1

The correlation matrix is simply a table of correlations.Each random variable in the table is correlated with each of the other values in the table. This allows us to see which pairs have the highest correlation.It also tells us if it is directly dependent or inversely by signifying the positive and negative signs.

In [ ]:
# Calculating correlation matrix for all variables
dfpd1.corr()

Representing the same matrix in form of heatmap where shades of blue signify inverse relation and shades of red signify direct relation.

In [ ]:
# Representing Matrix as a plot
from IPython.core.pylabtools import figsize

f, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(
    dfpd1.corr(),
    center=0,
    linewidths=0.8,
    cmap="coolwarm",
    annot=True,
    annot_kws={"size": 9},
)
plt.title("Variable Correlation")

Plotting all variables against other variables to see if relation exists between them. Basically it tells the same thing as correlation matrix but in a graphical way.

In [ ]:
sns.pairplot(dfpd1)

When all independent variables were plotted against 'price' variable it is observed that for'Landsize' and 'BuildingArea' roughly a linear line is obtained. So it can be concluded from this plot that variables Landsize and BuildingArea have linear relation with Price

##**H20 AutoML Rexecution on new model**

Dropping the variables that are not significant for determining Price. Passing this new model again through H20AutoML.

In [ ]:
df1 = df.drop(["Date", "Address", "Bedroom2", "Longtitude", "YearBuilt"], axis=1)

In [ ]:
df1_train, df1_test = df1.split_frame([pct_rows])

In [ ]:
X1 = df1.columns
print(X1)

In [ ]:
# Seperate Dependent variable from Independent variable
y1_numeric = "Price"
X1.remove(y1_numeric)
print(X1)

In [ ]:
aml1 = H2OAutoML(max_runtime_secs=run_time, seed=1)

In [ ]:
aml1.train(x=X1, y=y1_numeric, training_frame=df1_train)

In [ ]:
print(aml1.leaderboard)

In [ ]:
# assign index values to all the models generated
model_index = 0
glm_index = 0
glm_model = ""
aml1_leaderboard_df1 = aml1.leaderboard.as_data_frame()
models_dict = {}
for m in aml1_leaderboard_df1["model_id"]:
    models_dict[m] = model_index
    if "StackedEnsemble" not in m:
        break
    model_index = model_index + 1

for m in aml1_leaderboard_df1["model_id"]:
    if "GLM" in m:
        models_dict[m] = glm_index
        break
    glm_index = glm_index + 1
models_dict

NameError: ignored

In [ ]:
# print the index value of best model
print(model_index)
best_model1 = h2o.get_model(aml1.leaderboard[model_index, "model_id"])

In [ ]:
best_model1.algo

In [ ]:
# plot variables in order of their importance for price prediction
if best_model1.algo in ["gbm", "drf", "xrt", "xgboost"]:
    best_model1.varimp_plot()

In [ ]:
if glm_index is not 0:
    print(glm_index)
    glm_model1 = h2o.get_model(aml1.leaderboard[glm_index, "model_id"])
    print(glm_model1.algo)
    glm_model1.std_coef_plot()

In [ ]:
best_model1.partial_plot(
    df1,
    cols=[
        "BuildingArea",
        "Regionname",
        "Lattitude",
        "Postcode",
        "Landsize",
        "Distance",
        "Rooms",
        "Suburb",
    ],
)

##**Checking if assumptions violated**

*   列表项
*   列表项



Checking if the model has violated any assumptions. For linear regression some of the assumptions are: (1)There needs to be linear relation between the independent and dependent variable. (2)Homoscedasticity which means normality of the error distribution.The plot for residuals against target variable should be normally distributed. (3)Thirdly, linear regression assumes that there is little or no multicollinearity in the data. Multicollinearity occurs when the independent variables are too highly correlated with each other. (4)There should be no or little auto correlation. One way to test this is by Durbin-Watson test. The values should be brtween 1.5-2.5 for no autocorrelation.

In [ ]:
A = dfpd1.drop(["Price"], axis=1)
B = dfpd1["Price"]

In [ ]:
A_train, A_test, b_train, b_test = tts(A, B, test_size=0.2, random_state=42)

In [ ]:
cols1 = [
    "Suburb",
    "Rooms",
    "Type",
    "Method",
    "SellerG",
    "Distance",
    "Postcode",
    "Bathroom",
    "Car",
    "Landsize",
    "BuildingArea",
    "CouncilArea",
    "Lattitude",
    "Regionname",
    "Propertycount",
]
model1 = sm.OLS(b_train, sm.add_constant(A_train[cols1])).fit()

In [ ]:
b_pred = model1.predict(sm.add_constant(A_train[cols1]))

In [ ]:
residuals = b_train - b_pred
mean_residuals = np.mean(residuals)
print("Mean of Residuals {}".format(mean_residuals))

In [ ]:
p = sns.distplot(residuals, kde=True)
p = plt.title("Normality of error terms/residuals")

Ideally for the scatterplot the result should be scattered and it should not have any particular pattern.When the same is plotted as distplot it should follow a bell curve pattern.Both the plots give the expected result so even this assumptions are satisfied.



In [ ]:
import pylab
import scipy.stats as stats

stats.probplot(residuals, dist="norm", plot=pylab)
pylab.show()

In [ ]:
sns.histplot(dfpd.Price, kde=True)

##**Ridge Regularization in H20**

1.   列表项
2.   列表项




The main purpose of regression is to find patterns in this underlying dataset and generalize it to predict the corresponding target value for some new values of independent values. But sometimes target dataset is inflicted with some random noise. Although there may not be error for training set it will give huge errors in predicting the correct target values for test dataset.To avoid this condition regularization is used. Regularization is a technique used for tuning the random noise function by adding an additional term to noise function. This additional term controls the excessively fluctuating function such that the coefficients don’t take extreme values and the prediction of target value for test data is not highly affected.
For this model, Ridge Regularization is used.

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

house_glm = H2OGeneralizedLinearEstimator(
    family="gaussian", lambda_=0, compute_p_values=True
)
house_glm_regularization = H2OGeneralizedLinearEstimator(
    family="gaussian", lambda_=0.001, alpha=0
)

In [ ]:
house_glm_regularization.train(x=X1, y=y1_numeric, training_frame=df1_train)

In [ ]:
# Model details without regularization
house_glm.train(x=X1, y=y1_numeric, training_frame=df1_train)

The accuracy and other related variables were same for both the cases:
When dataset was regularized and when dataset was not regularized. This tells that for this specific model regularization does not help.

Calculating p-value and Z-value after regularization

In [ ]:
coeff_table = house_glm._model_json["output"]["coefficients_table"]

In [ ]:
coeff_table.as_data_frame()

##**Data Report**

In [ ]:
exa = aml1.explain(df1_test)

##**Hyperparameter Tuning** <br>


To find best set a hyperparameter and combinations of interacting hyperparameters for a given dataset hyperparameters tuning is used. It objectively searches different values for model hyperparameters and chooses a subset that results in a model that achieves the best performance on a given dataset.

In [ ]:
s = dfpd1["Price"]

t = dfpd1.drop(["Price"], axis=1)

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
t_train, t_test, s_train, s_test = train_test_split(
    t, s, random_state=101, test_size=0.2
)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor

mode = RandomForestRegressor()

param_vals = {
    "max_depth": [200, 500, 800, 1100],
    "n_estimators": [100, 200, 300, 400],
    "min_samples_split": [2, 3, 5],
}

random_rf = RandomizedSearchCV(
    estimator=mode,
    param_distributions=param_vals,
    n_iter=10,
    scoring="accuracy",
    cv=5,
    refit=True,
    n_jobs=-1,
)

# Training and prediction


random_rf.fit(t_train, s_train)

preds = random_rf.best_estimator_.predict(t_test)

In [ ]:
random_rf.best_params_

##**Conclusion**

A fundamental algorithm based on the linear regression method to predict housing prices was constructed. VIF, p-values and other tests were considered to ignore some of the independent variables.  H20.ai framework was used to train and test the variables of the dataset of Melbourne Housing price. This gave the best model as ‘gbm’ and accuracy for test data was 89.9%.  From the results shown it can be concluded that the proposed linear regression model can effectively analyze and predict the housing price to some extent. Admittedly, the prediction accuracy is still limited at specific points, and the model still needs to be improved in further research. In further research into the corresponding models, different techniques like removing outliers and using ensemble or booster can help achieve better prediction.

##**Assignment Question/Answers**



Q1) Is the relationship significant?
A relationship is said to be statistically significant when the p value for the variables is less than 0.05.P value is probability of obtaining a result at least as extreme as the current one, assuming null hypothesis is true. A small p value depicts that there is very little similarity between the two groups and hence null hypothesis can be ignored.
P-value was calculated by two ways for this model. From the OLS method it was observed that values for ‘Address’ and ‘Date’ were greater than 0.05. So, it can be concluded that other that p-value for all the variables of dataset is less than 0.05. 
Hence for the chosen dataset it can be concluded that the relationship is significant.



Q2) Are any model assumptions violated?
Assumptions for linear regression are:
1)	Linear relationship- The graph for dependent and independent variable needs to be linear by keeping other variables constant. When target variable is plotted against all other independent variables, linear relation is observed for few of them. Hence this assumption is not violated.
2)	Homoscedasticity which means normality of the error distribution - The plot for residuals should be normally distributed i.e., it should form a bell-curve shape. For this model the same is achieved.
3)	No or little multicollinearity-This can be determined by correlation matrix. For this model some of the assumptions are violated. Multicollinearity exists between variables Rooms and BuildingArea. 
4)	No autocorrelation can be determined with Durbin-Watson test. Values between 1.5 to 2.5 signifies no autocorrelation. For this model the value is 1.6, therefore this assumption is not violated.




Q3) Is there any multicollinearity in the model?
When one independent variable is related to another independent variable that is its highly correlated then multicollinearity exists in the model. It’s an issue as it undermines the statistical significance of an independent variable.
Multicollinearity can be determined by computing correlation matrix or calculating VIF value of all variables.
In correlation matrix a coefficient closer to +1 or -1 tells us that those two variables are highly correlated. For VIF values a value above 10 tells us that multicollinearity exists.
In this model some variables had VIF value greater than 10 and p value was greater than 0.05. By dropping those columns and rechecking the values expected output was achieved. Currently high correlation exists between Rooms and BuildingArea 



Q4) In the multivariate models are predictor variables independent of all the other predictor variables?
Variables are said to be independent when there is no relation between them. To check this relation, correlation matrix can be used, or it can be observed from graphs too whether there is any pattern followed or not.
When correlation matrix is computed for the model, it can be observed that Rooms and BuildingArea are correlated to each other. Other than those other predictors are independent from each other.



Q5)In in multivariate models rank the most significant predictor variables and exclude insignificant ones from the model.
From the variable importance plot, the most to least important variables are displayed.
For my model BuildingArea, Type, Latitude and Rooms are top 4 most important variables to determine Price. VIF and p-values for Address, Date, Bedroom2, YearBuilt and Longtitude was high than the ideal values. So those variables were excluded from the model.



Q6) Does the model make sense?
For a model to make sense it should follow all the assumptions and have p value, VIF between their respective ranges. RMSE should be as low as possible considering the minimum and maximum values of the target variable. Other than that, R2 too is 0.76 which is considered good in terms of accuracy. So overall the model makes sense.
To increase the accuracy, some additional variables can be dropped depending on their importance. Furthermore, outliers can be removed or boosting, or ensemble model can be used.



Q7)Does regularization help?
Regularization is a technique used for tuning the random noise function by adding an additional term to noise function. This additional term controls the excessively fluctuating function such that the coefficients don’t take extreme values and the prediction of target value for test data is not highly affected. The main use of Regularization is to minimize the validation loss and try to improve the accuracy of the model.
For this model Ridge Regularization was used on training data. It was observed that Root Mean Square Error (RMSE) and R2 was calculated twice, once when regularization was not applied and once when regularization was applied. The values were same in both the cases. Hence it can be concluded that for this model regularization does not help.



Q8) Which independent variables are significant?
 Variables are significant when p-value is less than 0.05. For this model except for ‘Date’,’YearBuilt’ and ‘Address’ all other variables have p-value less than 0.05. So, it can be said that all variables are significant.



Q9) Which hyperparameters are important?
To find best set a hyperparameter and combinations of interacting hyperparameters for a given dataset hyperparameters tuning is used. It objectively searches different values for model hyperparameters and chooses a subset that results in a model that achieves the best performance on a given dataset.
For this model tuning is performed using RandomForestRegressor. The best hyperparameters for this model are:-
'max_depth': 500, 'min_samples_split': 2 and 'n_estimators': 100

##**LICENSE**

MIT License

Copyright (c) 2022 Sanika Patne

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

##**REFERENCES**

H20.ai- https://docs.h2o.ai/ <br>
OLS Model- http://net-informations.com/ds/mla/ols.html<br>
Linear Regression- https://www.analyticsvidhya.com/blog/2021/05/all-you-need-to-know-about-your-first-machine-learning-model-linear-regression/ <br>
Linear Regression Assumptions- https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/assumptions-of-linear-regression/ <br>
Kaggle Notebook- https://www.kaggle.com/stephaniestallworth/melbourne-housing-market-eda-and-regression <br>
Dataset- https://www.kaggle.com/dansbecker/melbourne-housing-snapshot <br>
Professor's AutoML Notebook- https://github.com/nikbearbrown/AI_Research_Group/tree/main/Kaggle_Datasets/AutoML